In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import random
import math
import time
from torchtext import data
from transformers import T5Tokenizer, T5Model

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [4]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [5]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [6]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [7]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [8]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [9]:
SRC = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TRG = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

In [10]:
fields = [('src', SRC), ('trg', TRG)]

In [11]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'marco2.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(4321))

In [12]:
print(len(train_data.examples))
print(len(valid_data.examples))

88200
1800


In [13]:
print(vars(train_data.examples[10000]))

{'src': [2625, 3, 10, 8, 3334, 1085, 1104, 1080, 21, 3, 4059, 109, 4411, 6, 3, 7, 75, 19, 4848, 2712, 5, 48, 19, 8, 792, 13, 538, 6, 5435, 11, 690, 1085, 1104, 1917, 5, 8, 3414, 443, 12057, 9, 538, 1085, 1104, 1080, 19, 1083, 3, 6370, 5, 8, 3, 1152, 1050, 1306, 5435, 1085, 1104, 1080, 19, 3, 6932, 5, 3, 4059, 109, 4411, 1085, 1104, 1080, 19, 3, 4704, 5, 11417, 3, 10, 125, 19, 8, 1085, 1104, 1080, 16, 3, 4059, 109, 4411, 3, 7, 75], 'trg': [4848, 2712]}


In [14]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10000])['src'])

print(tokens)
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10000])['trg'])

print(tokens)

['▁context', '▁', ':', '▁the', '▁combined', '▁sales', '▁tax', '▁rate', '▁for', '▁', 'char', 'le', 'ston', ',', '▁', 's', 'c', '▁is', '▁8.', '5%', '.', '▁this', '▁is', '▁the', '▁total', '▁of', '▁state', ',', '▁county', '▁and', '▁city', '▁sales', '▁tax', '▁rates', '.', '▁the', '▁south', '▁car', 'olin', 'a', '▁state', '▁sales', '▁tax', '▁rate', '▁is', '▁currently', '▁', '6%', '.', '▁the', '▁', 'ber', 'ke', 'ley', '▁county', '▁sales', '▁tax', '▁rate', '▁is', '▁', '0%', '.', '▁', 'char', 'le', 'ston', '▁sales', '▁tax', '▁rate', '▁is', '▁', '1%', '.', '▁query', '▁', ':', '▁what', '▁is', '▁the', '▁sales', '▁tax', '▁rate', '▁in', '▁', 'char', 'le', 'ston', '▁', 's', 'c']
['▁8.', '5%']


In [15]:
device = torch.device('cuda')

BATCH_SIZE = 20

train_iterator, valid_iterator = BucketIterator.splits(
                                (train_data, valid_data), 
                                batch_size = BATCH_SIZE,
                                device = device,
                                sort_key=lambda x: len(x.src))

In [16]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(t5.config.to_dict()['d_model'], t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src, decoder_input_ids=trg)
        
        output = self.out(embedded[0])
        
        return output

In [17]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [19]:
LEARNING_RATE = 0.0004

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [20]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_token_idx)

In [21]:
N_EPOCHS = 4
CLIP = 1

In [22]:
for epoch in range(N_EPOCHS):
    
    start = time.time()
    # TRAIN 
    #########################################################################
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(train_iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg[:,:-1])
        
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    train_loss = epoch_loss / len(train_iterator)
    #########################################################################
    
    # VALID
    #########################################################################
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(valid_iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg[:,:-1])
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    valid_loss = epoch_loss / len(valid_iterator)
    #########################################################################
    end = time.time()
    
    print(f"EPOCH : {epoch+1}\tTRAIN LOSS : {train_loss:.2f}\tVALID LOSS : {valid_loss:.2f}\tTIME : {end-start:.2f}\n")
    torch.save(model.state_dict(), f'saved_models/marco_model_{epoch+1}.pt')

EPOCH : 1	TRAIN LOSS : 1.96	VALID LOSS : 0.83	TIME : 1413.11

EPOCH : 2	TRAIN LOSS : 0.73	VALID LOSS : 0.72	TIME : 1442.93

EPOCH : 3	TRAIN LOSS : 0.56	VALID LOSS : 0.71	TIME : 1443.43

EPOCH : 4	TRAIN LOSS : 0.47	VALID LOSS : 0.74	TIME : 2776.08



In [23]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break
            
    return trg_indexes[1:]

In [24]:
idxs = random.sample(range(0,len(valid_data.examples)),20)

for i in idxs:
    src = vars(valid_data.examples[i])['src']
    trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence(src, SRC, TRG, model, device)

    print(f"SRC : {' '.join(tokenizer.convert_ids_to_tokens(src))}")
    print(f"TRG : {' '.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PREDICTED : {' '.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context ▁ : ▁add ▁ a ▁photo ▁to ▁this ▁gallery . ▁1 ▁them y s cir a ▁was ▁shown ▁on ▁the ▁television ▁movie ▁wonder ▁woman ▁ starring ▁actress ▁ca th y ▁le e ▁ cro s by . ▁the ▁scene ▁was ▁very ▁brief ▁and ▁no ▁details ▁about ▁the ▁island ▁were ▁disclosed . ▁ titled ▁paradise ▁island , ▁it ▁was ▁shown ▁in ▁the ▁ ly nd a ▁carte r ▁television ▁series ▁wonder ▁woman ▁throughout ▁several ▁episodes . ▁query ▁ : ▁what ▁island ▁was ▁used ▁for ▁them y s cir a ▁in ▁wonder ▁woman
TRG : ▁paradise ▁island
PREDICTED : ▁paradise ▁island </s>

SRC : ▁context ▁ : ▁nutrition ▁summary : ▁there ▁are ▁130 ▁calories ▁in ▁ a ▁1 ▁cup ▁serving ▁of ▁healthy ▁choice ▁italian ▁style ▁wedding ▁soup ▁( m eat ball s ▁ & ▁spinach ▁in ▁chicken ▁broth ▁with ▁ a cini ▁de ▁pe pe ▁pasta ). ▁ calorie ▁breakdown : ▁1 6% ▁fat , ▁5 5% ▁carb s , ▁2 9% ▁protein . ▁query ▁ : ▁calories ▁in ▁wedding ▁soup ▁broth
TRG : ▁130 ▁calories
PREDICTED : ▁130 ▁calories ▁in ▁ a ▁1 ▁cup </s>

SRC : ▁context ▁ : ▁the ▁city ▁had ▁ ... ▁